In [4]:
import sys
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

sys.path.append(os.path.abspath('..')) 

from src.data_loader import load_data, clean_data

df = load_data()
df = clean_data(df)


In [7]:
# Przeskalowanie danych, aby wszystkie cechy miały taką samą wagę
scaler = StandardScaler()
features_scale = scaler.fit_transform(df[['stars', 'reviews', 'price']])

def recommend_products(asin, top_n=5):
    if asin not in df['asin'].values:
        return f"Produkt o ASIN {asin} nie istnieje."
    
    # Znalezienie indeksy produktu i kategorii
    product_idx = df.index[df['asin'] == asin][0]
    category = df.loc[product_idx, 'categoryName']
    
    # Filtrowanie produktów z tej samej kategorii
    category_mask = df['categoryName'] == category
    category_features = features_scale[category_mask]
    category_indices = df.index[category_mask]
    
    # Wektor cech produktu
    product_vector = features_scale[product_idx].reshape(1, -1)
    
    # Obliczenie podobieństwa
    similarity = cosine_similarity(product_vector, category_features).flatten()
    
    # Wykluczenie produkty samego z siebie
    similarity[category_indices.get_loc(product_idx)] = -1
    
    # Pobranie indeksów najbardziej podobnych produktów
    top_indices = similarity.argsort()[-top_n:][::-1]
    
    # Mapuj do oryginalnych indeksów DataFrame
    recommended_indices = category_indices[top_indices]
    
    return df.loc[recommended_indices, ['asin', 'title', 'stars', 'reviews', 'price', 'categoryName']]

In [8]:
# Przykład użycia
print(recommend_products('B09B96TG33', 5))

           asin                                              title  stars  \
2    B09B8YWXDF  Echo Dot (5th generation, 2022 release) | Big ...    4.7   
6    B09B9615X2  Echo Dot (5th generation, 2022 release) | Big ...    4.7   
508  B0BYPHGSMK  Echo Dot (5th generation, 2022 release), Glaci...    4.7   
324  B0BYPHCV5L  Echo Dot (5th generation, 2022 release), Deep ...    4.7   
217  B0BYPJ1GPZ  Echo Dot (5th generation, 2022 release), Charc...    4.7   

     reviews  price    categoryName  
2      15308  21.99  Hi-Fi Speakers  
6      15308  21.99  Hi-Fi Speakers  
508    15308  28.99  Hi-Fi Speakers  
324    15308  28.99  Hi-Fi Speakers  
217    15308  28.99  Hi-Fi Speakers  
